# Подготовка данных в рамках для решения задачи определения социальных характеристик пользователей от компании RUTUB

In [1]:
# импорт необходимых библиотек
import datetime
import pandas as pd
import numpy as np
import statistics
from sklearn.preprocessing import OrdinalEncoder


In [2]:
def get_pivot_data(df, name_pivot, name_values, top_df=None, name_features=None, other_name=None):
    '''Создает фичи путем преобразования значений в столбцы и заоплнением полезной информацией

    '''
    data = df.copy()
    if top_df is not None:
        video_dict = {i: i for i in top_df.index}
        data[name_pivot] = data[name_features]
        data[name_pivot] = data[name_pivot].map(video_dict)
        data[name_pivot] = data[name_pivot].fillna(other_name)

        data = data.drop(name_features, axis=1)

    df_res = pd.pivot_table(
        data,
        values=[name_values],
        index=['viewer_uid'],
        columns=[name_pivot],
        aggfunc=['sum']
        )

    df_res.columns = df_res.columns.get_level_values(2)
    df_res = df_res.reset_index()
    df_res = df_res.fillna(0)
    return df_res


In [3]:
df = pd.read_csv('data/all_events.csv')
#трейн датасет
data = pd.read_csv('data/train_events.csv')
df_target = pd.read_csv('data/train_targets.csv')
# загрузка данных с временными зонами регионов
region_tz = pd.read_csv("data/region_tz.csv")
# загрузка текстовых данных
df_video = pd.read_csv('data/video_info_lemm.csv')
df_info_text = pd.read_csv('data/video_info_text_count.csv')
df_time_data = pd.read_csv('data/data.csv')
df.shape, data.shape, df_target.shape, df_time_data.shape


((8439624, 9), (1759616, 9), (180012, 4), (1759616, 26))

In [4]:
# меняем пропуски на популярное время суток
df_time_data = df_time_data.fillna('вечер')
# Меняем пропуски на Unknown
data['ua_os'] = data['ua_os'].fillna('Unknown')


In [5]:
dict_time_cut = {
    'утро' : 'morning',
    'вечер' : 'evening',
    'ночь' : 'night',
    'день' : 'day'
}
df_time_data['time_cut'] = df_time_data['time_cut'].map(dict_time_cut)
# удаляем лишнии столбцы
df_time_data = df_time_data[['event_timestamp', 'viewer_uid', 'timezone', 'lat', 'lon', 'time_cut', 'current_weekday', 'is_day_off']]


In [6]:
# Соединяем датасеты
data = data.merge(df_time_data, on=['event_timestamp','viewer_uid'], how='inner')


In [7]:
del df_time_data


In [8]:
df_video['duration'] = df_video['duration'] / 1000 / 60
df_video['duration'] = df_video['duration'].astype('float32')
# категориальный признак длительности видео
cut_minuts = ["short_video", "means_video", "long_video", "very_long_video"]
cut_bins = [0, 50, 100, 600, 1500]

df_video["duration_qcut"] = pd.cut(df_video["duration"],
                              bins=cut_bins,
                              labels=cut_minuts)
df_video


,rutube_video_id,title,category,duration,author_id,minuts,minuts qcut,text,lemm_text_list,lemm_text,duration_qcut
0,video_185549,Как собрать букет из мыльных тюльпанов - Силик...,Хобби,25.986000,1015054,25.986000,короткие,как собрать букет из мыльных тюльпанов силик...,"['как', 'собирать', 'букет', 'из', 'мыльный', ...",как собирать букет из мыльный тюльпан силиконо...,short_video
1,video_111035,"Осторожно, Киберземляне!, 1 сезон, 12 серия",Сериалы,22.000116,1002180,22.000117,короткие,осторожно киберземляне сезон серия,"['осторожно', 'киберземлянин', 'сезон', 'серия']",осторожно киберземлянин сезон серия,short_video
2,video_476517,ПОПУЛЯРНЫЕ ВИДЕОИГРЫ в LEGO... перевод - TD BR...,Хобби,10.102417,1095337,10.102417,короткие,популярные видеоигры в перевод ...,"['популярный', 'видеоигра', 'в', 'перевод']",популярный видеоигра в перевод,short_video
3,video_157198,"Хороший лжец (фильм, 2019)",Фильмы,109.624001,1043618,109.624000,продолжительные,хороший лжец фильм,"['хороший', 'лжец', 'фильм']",хороший лжец фильм,long_video
4,video_289824,Нашего старого гнобят по-всякому,Развлечения,14.324883,1009535,14.324883,короткие,нашего старого гнобят по всякому,"['наш', 'старый', 'гнобить', 'по', 'всякий']",наш старый гнобить по всякий,short_video
...,...,...,...,...,...,...,...,...,...,...,...
481475,video_196126,14. Landscape Layer Blend,Дизайн,7.700000,1121338,7.700000,короткие,,[],NaN,short_video
481476,video_390909,Рихард Штраус: Завтра! — Людмила Рубинская,Музыка,3.793800,1043510,3.793800,короткие,рихард штраус завтра людмила рубинская,"['рихард', 'штраус', 'завтра', 'людмила', 'руб...",рихард штраус завтра людмила рубинский,short_video
481477,video_326364,"Поймай Тинипин! Королевство эмоций, 2 серия. С...",Мультфильмы,12.409600,1002799,12.409600,короткие,поймай тинипин королевство эмоций серия с...,"['поймать', 'тинипин', 'королевство', 'эмоция'...",поймать тинипин королевство эмоция серия самый...,short_video
481478,video_374957,ВОРКУТА. 1 СЕРИЯ. УЖАСЫ. ПСИХОЛОГИЧЕСКИЙ ТРИЛЛЕР,Люди и блоги,43.289749,1008386,43.289750,короткие,воркута серия ужасы психологический триллер,"['воркута', 'серия', 'ужас', 'психологический'...",воркута серия ужас психологический триллер,short_video


In [9]:
# Расчитываем сквозные показатели
data['cnt_views'] = data.groupby(['rutube_video_id', 'viewer_uid'])['event_timestamp'].transform('count')
df['cnt_views'] = df.groupby(['rutube_video_id', 'viewer_uid'])['event_timestamp'].transform('count')
data


,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,timezone,lat,lon,time_cut,current_weekday,is_day_off,cnt_views
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243,5,55.1644,61.4368,morning,5,True,1
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341,5,54.7388,55.9721,evening,5,True,1
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333,3,59.9343,30.3351,evening,5,True,1
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092,3,55.7558,37.6173,evening,5,True,2
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976,3,55.7558,37.6173,evening,5,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759751,2024-06-30 17:08:36+03:00,Moscow,smartphone,mobile app,Android,Rutube,4230,video_162776,10026914,3,55.7558,37.6173,day,6,True,2
1759752,2024-06-30 01:20:16+03:00,Moscow,smartphone,browser,Android,Firefox Mobile,382,video_316157,10417567,3,55.7558,37.6173,night,6,True,1
1759753,2024-06-30 22:37:04+03:00,St.-Petersburg,desktop,browser,Windows,Microsoft Edge,1342,video_83304,10009094,3,59.9343,30.3351,evening,6,True,1
1759754,2024-06-30 11:33:09+03:00,St.-Petersburg,smartphone,mobile app,Android,Rutube,801,video_132769,10574374,3,59.9343,30.3351,morning,6,True,1


In [11]:
data['total_watchtime'] = data['total_watchtime'] / 60
df['total_watchtime'] = df['total_watchtime'] / 60
data['total_watchtime'] = data['total_watchtime'].astype('int32')
df['total_watchtime'] = df['total_watchtime'].astype('int32')
data['cnt_views'] = data['cnt_views'].astype('int32')
data['viewer_uid'] = data['viewer_uid'].astype('int32')
df['cnt_views'] = df['cnt_views'].astype('int32')
df['viewer_uid'] = df['viewer_uid'].astype('int32')


In [12]:
all_df = pd.concat([df, data])
all_df


,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,cnt_views,timezone,lat,lon,time_cut,current_weekday,is_day_off
0,2024-06-01 13:08:30+03:00,Tatarstan Republic,smartphone,browser,Android,Chrome Mobile,1,video_395879,10813370,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-06-01 14:30:00+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,1,video_216518,10512324,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-06-01 18:48:12+03:00,Novosibirsk Oblast,smartphone,mobile app,Android,Rutube,2,video_41225,10951137,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-01 16:32:36+03:00,Moscow,desktop,browser,Windows,Yandex Browser,38,video_215886,10912434,1,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-06-01 00:57:04+03:00,Moscow Oblast,smartphone,mobile app,Android,Rutube,113,video_43631,10223585,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759751,2024-06-30 17:08:36+03:00,Moscow,smartphone,mobile app,Android,Rutube,70,video_162776,10026914,2,3.0,55.7558,37.6173,day,6.0,True
1759752,2024-06-30 01:20:16+03:00,Moscow,smartphone,browser,Android,Firefox Mobile,6,video_316157,10417567,1,3.0,55.7558,37.6173,night,6.0,True
1759753,2024-06-30 22:37:04+03:00,St.-Petersburg,desktop,browser,Windows,Microsoft Edge,22,video_83304,10009094,1,3.0,59.9343,30.3351,evening,6.0,True
1759754,2024-06-30 11:33:09+03:00,St.-Petersburg,smartphone,mobile app,Android,Rutube,13,video_132769,10574374,1,3.0,59.9343,30.3351,morning,6.0,True


In [13]:
del df


In [14]:
data


,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,timezone,lat,lon,time_cut,current_weekday,is_day_off,cnt_views
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,31,video_133074,10067243,5,55.1644,61.4368,morning,5,True,1
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,8,video_362960,10245341,5,54.7388,55.9721,evening,5,True,1
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,94,video_96775,10894333,3,59.9343,30.3351,evening,5,True,1
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,25,video_161610,10029092,3,55.7558,37.6173,evening,5,True,2
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,1,video_116245,10452976,3,55.7558,37.6173,evening,5,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759751,2024-06-30 17:08:36+03:00,Moscow,smartphone,mobile app,Android,Rutube,70,video_162776,10026914,3,55.7558,37.6173,day,6,True,2
1759752,2024-06-30 01:20:16+03:00,Moscow,smartphone,browser,Android,Firefox Mobile,6,video_316157,10417567,3,55.7558,37.6173,night,6,True,1
1759753,2024-06-30 22:37:04+03:00,St.-Petersburg,desktop,browser,Windows,Microsoft Edge,22,video_83304,10009094,3,59.9343,30.3351,evening,6,True,1
1759754,2024-06-30 11:33:09+03:00,St.-Petersburg,smartphone,mobile app,Android,Rutube,13,video_132769,10574374,3,59.9343,30.3351,morning,6,True,1


In [15]:
# Топ популярных видео в признаки
top_video = all_df['rutube_video_id'].value_counts().to_frame()
top_video = top_video[top_video['count'] > 1500]
display('top_video len -', len(top_video))

df_final = get_pivot_data(
    df=data,
    name_pivot='top_video',
    name_values='cnt_views',
    top_df=top_video,
    name_features='rutube_video_id',
    other_name='other_video',
)
df_final


'top_video len -'

814

top_video,viewer_uid,other_video,video_101699,video_102608,video_102934,video_10383,video_104009,video_104702,video_10697,video_109515,...,video_95009,video_95581,video_96465,video_96775,video_97997,video_98149,video_98194,video_98702,video_98905,video_9992
0,10000001,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000002,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10000005,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000006,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180007,11140828,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180008,11140869,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180009,11140872,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180010,11140875,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
data = data.merge(df_video, on='rutube_video_id', how='inner')
data.shape


(1759756, 26)

In [17]:
video_df_merge = all_df.merge(df_video, on='rutube_video_id', how='inner')
video_df_merge


,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,cnt_views,...,title,category,duration,author_id,minuts,minuts qcut,text,lemm_text_list,lemm_text,duration_qcut
0,2024-06-01 13:08:30+03:00,Tatarstan Republic,smartphone,browser,Android,Chrome Mobile,1,video_395879,10813370,1,...,ТАЙНА ДВОРЦОВОГО МОСТА В ПЕТЕРБУРГЕ | А ЧТО ПО...,Лайфстайл,5.555217,1012572,5.555217,короткие,тайна дворцового моста в петербурге а что по...,"['тайна', 'дворцовый', 'мост', 'в', 'петербург...",тайна дворцовый мост в петербург а что под мост,short_video
1,2024-06-01 14:30:00+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,1,video_216518,10512324,1,...,"Я художник, я так вижу. Мужское / Женское. Вып...",Телепередачи,41.671234,1009219,41.671233,короткие,я художник я так вижу мужское женское вып...,"['я', 'художник', 'я', 'так', 'видеть', 'мужск...",я художник я так видеть мужской женский выпуск от,short_video
2,2024-06-01 18:48:12+03:00,Novosibirsk Oblast,smartphone,mobile app,Android,Rutube,2,video_41225,10951137,1,...,ДОРАМЫ ПРО ЛЮБОВЬ ♥ Романтические Фильмы ♥ Кор...,Фильмы,4.299950,1023998,4.299950,короткие,дорамы про любовь романтические фильмы кор...,"['дорам', 'про', 'любовь', 'романтический', 'ф...",дорам про любовь романтический фильм корейский...,short_video
3,2024-06-01 16:32:36+03:00,Moscow,desktop,browser,Windows,Yandex Browser,38,video_215886,10912434,1,...,Менталист / The Mentalist – 4 сезон 13 серия,Сериалы,39.389885,1074854,39.389883,короткие,менталист сезон серия,"['менталист', 'сезон', 'серия']",менталист сезон серия,short_video
4,2024-06-01 00:57:04+03:00,Moscow Oblast,smartphone,mobile app,Android,Rutube,113,video_43631,10223585,1,...,Сериал Тёмная материя – 1 сезон 4 серия / Dark...,Сериалы,59.822933,1036095,59.822933,средние,сериал тёмная материя сезон серия ...,"['сериал', 'темный', 'материя', 'сезон', 'серия']",сериал темный материя сезон серия,means_video
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10199375,2024-06-30 17:08:36+03:00,Moscow,smartphone,mobile app,Android,Rutube,70,video_162776,10026914,2,...,"Я знаю, но 1 сезон 3 серия",Сериалы,74.681915,1095392,74.681917,средние,я знаю но сезон серия,"['я', 'знать', 'но', 'сезон', 'серия']",я знать но сезон серия,means_video
10199376,2024-06-30 01:20:16+03:00,Moscow,smartphone,browser,Android,Firefox Mobile,6,video_316157,10417567,1,...,РЕАКЦИЯ ЮРИСТА НА TikTok | НЕАДЕКВАТНЫЕ СОТРУД...,Обучение,5.335567,1033688,5.335567,короткие,реакция юриста на неадекватные сотруд...,"['реакция', 'юрист', 'на', 'неадекватный', 'со...",реакция юрист на неадекватный сотрудник в метро,short_video
10199377,2024-06-30 22:37:04+03:00,St.-Petersburg,desktop,browser,Windows,Microsoft Edge,22,video_83304,10009094,1,...,Свои чужие дети. Мужское / Женское. Выпуск от ...,Телепередачи,52.094582,1009219,52.094583,средние,свои чужие дети мужское женское выпуск от ...,"['свой', 'чужой', 'ребенок', 'мужской', 'женск...",свой чужой ребенок мужской женский выпуск от,means_video
10199378,2024-06-30 11:33:09+03:00,St.-Petersburg,smartphone,mobile app,Android,Rutube,13,video_132769,10574374,1,...,"Коля, Коля, Николай. Мужское / Женское. Выпуск...",Телепередачи,40.021385,1009219,40.021383,короткие,коля коля николай мужское женское выпуск...,"['коля', 'коля', 'николай', 'мужской', 'женски...",коля коля николай мужской женский выпуск от,short_video


In [18]:
video_df_merge.isna().sum()


event_timestamp          0
region                   0
ua_device_type           0
ua_client_type          13
ua_os               519630
ua_client_name          13
total_watchtime          0
rutube_video_id          0
viewer_uid               0
cnt_views                0
timezone           8439624
lat                8439624
lon                8439624
time_cut           8439624
current_weekday    8439624
is_day_off         8439624
title                    0
category                 0
duration                 0
author_id                0
minuts                   0
minuts qcut              0
text                     0
lemm_text_list           0
lemm_text            26676
duration_qcut            0
dtype: int64

In [19]:
# Доля просмотра от общего видео
data['share_watchtime_duration'] = data['cnt_views'] / data['duration']
# топ популярных авторов
top_authtor = video_df_merge['author_id'].value_counts().to_frame()
top_authtor = top_authtor[top_authtor['count'] > 100]
display('top_authtor len -', len(top_authtor))

df_final_2 = get_pivot_data(
    df=data,
    name_pivot='top_authtor',
    name_values='share_watchtime_duration',
    top_df=top_authtor,
    name_features='author_id',
    other_name='other_author',
)
df_final_2


'top_authtor len -'

1551

top_authtor,viewer_uid,1000004.0,1000005.0,1000008.0,1000012.0,1000014.0,1000015.0,1000027.0,1000039.0,1000041.0,...,1123975.0,1124234.0,1124384.0,1124447.0,1124476.0,1124501.0,1124767.0,1124879.0,1124992.0,other_author
0,10000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,10000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,10000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,10000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.114266
4,10000006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180007,11140828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
180008,11140869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
180009,11140872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
180010,11140875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [20]:
df_final = df_final.merge(df_final_2, on='viewer_uid', how='inner')
df_final.shape


(180012, 2325)

In [21]:
# Заменяем пропуски в тексте категориями
data['lemm_text'] = data['lemm_text'].fillna(data['category'])


In [22]:
del all_df
del video_df_merge


In [23]:
categories_en = {
    'Телепередачи': 'Television Shows',
    'Юмор': 'Comedy',
    'Разное': 'Various',
    'Путешествия': 'Travel',
    'Сериалы': 'Serials',
    'Мультфильмы': 'Animated Movies',
    'Спорт': 'Sports',
    'Здоровье': 'Health',
    'Детям': 'For Kids',
    'Фильмы': 'Movies',
    'Лайфстайл': 'Lifestyle',
    'Видеоигры': 'Video Games',
    'Интервью': 'Interviews',
    'Развлечения': 'Entertainment',
    'Авто-мото': 'Cars and Motorcycles',
    'Технологии и интернет': 'Technology and Internet',
    'Наука': 'Science',
    'Обучение': 'Education',
    'Техника и оборудование': 'Equipment and Technology',
    'Аниме': 'Anime',
    'Эзотерика': 'Esoterica',
    'Культура': 'Culture',
    'Музыка': 'Music',
    'Бизнес и предпринимательство': 'Business and Entrepreneurship',
    'Аудиокниги': 'Audiobooks',
    'Обзоры и распаковки товаров': 'Product Reviews and Unboxing',
    'Еда': 'Food',
    'Природа': 'Nature',
    'Хобби': 'Hobbies',
    'Строительство и ремонт': 'Construction and Repair',
    'Дизайн': 'Design',
    'Красота': 'Beauty',
    'Сад и огород': 'Garden and Gardening',
    'Животные': 'Animals',
    'Аудио': 'Audio',
    'Лайфхаки': 'Lifehacks',
    'Психология': 'Psychology',
    'Люди и блоги': 'People and Blogs',
    'Охота и рыбалка': 'Hunting and Fishing',
    'Недвижимость': 'Real Estate'
}
# меняем на английский язык
data['category'] = data['category'].map(categories_en)


In [24]:
# Категории в столбцы
df_final_2 = get_pivot_data(
    df=data,
    name_pivot='category',
    name_values='total_watchtime',
)
df_final = df_final.merge(df_final_2, on='viewer_uid', how='inner')
df_final.shape


(180012, 2365)

In [25]:
# Время просмотра по бинам в столбцы
df_final_2 = get_pivot_data(
    df=data,
    name_pivot='duration_qcut',
    name_values='duration',
)
df_final = df_final.merge(df_final_2, on='viewer_uid', how='inner')
df_final.shape


/tmp/ipykernel_4940/1096783602.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  df_res = pd.pivot_table(


(180012, 2369)

In [26]:
# Время суток в столбцы
df_final_2 = get_pivot_data(
    df=data,
    name_pivot='time_cut',
    name_values='total_watchtime',
)
df_final = df_final.merge(df_final_2, on='viewer_uid', how='inner')
df_final.shape


(180012, 2373)

In [27]:
# День недели в столбцы
df_final_2 = get_pivot_data(
    df=data,
    name_pivot='current_weekday',
    name_values='cnt_views',
)
df_final = df_final.merge(df_final_2, on='viewer_uid', how='inner')
df_final.shape


(180012, 2380)

In [28]:
# Выходной день в столбцы
data['is_day_off'] = data['is_day_off'].map(lambda x: 'day off' if x == True else 'weekdays')
df_final_2 = get_pivot_data(
    df=data,
    name_pivot='is_day_off',
    name_values='cnt_views',
)
df_final = df_final.merge(df_final_2, on='viewer_uid', how='inner')
df_final.shape


(180012, 2382)

In [29]:
# Содеиденяем тектовые признаки
data = data.merge(df_info_text, on='rutube_video_id', how='inner')
data.head()


,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,timezone,...,lemm_text_list,lemm_text,duration_qcut,share_watchtime_duration,text_male_count,text_female_count,text_age_0_class_count,text_age_1_class_count,text_age_2_class_count,text_age_3_class_count
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,31,video_133074,10067243,5,...,"['папа', 'с', 'особенность', 'мужской', 'женск...",папа с особенность мужской женский выпуск от,short_video,0.024425,0,0,0,0,0,0
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,8,video_362960,10245341,5,...,"['мальдивы', 'андрей', 'бебуришвили']",мальдивы андрей бебуришвили,short_video,0.115560,0,0,0,0,0,0
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,94,video_96775,10894333,3,...,"['новый', 'битва', 'экстрасенс', 'сезон', 'вып...",новый битва экстрасенс сезон выпуск,means_video,0.010873,0,0,0,0,0,0
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,25,video_161610,10029092,3,...,"['сергей', 'орлов', 'снимать', 'дом']",сергей орлов снимать дом,short_video,0.078840,0,0,0,0,0,0
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,1,video_116245,10452976,3,...,"['искать', 'сокровище', 'в', 'полевский', 'ура...",искать сокровище в полевский уральский флоренц...,short_video,0.048003,0,0,0,0,0,0


In [30]:
%%time
# Грууппирему по юзеру
data_group = data.groupby(['viewer_uid']).agg(
    region_mode = ('region', lambda x: statistics.mode(list(x))),
    region_len = ('region', lambda x: len(set(x))),
    ua_device_type_mode = ('ua_device_type', lambda x: statistics.mode(list(x))),
    ua_device_type_len = ('ua_device_type', lambda x: len(set(x))),
    ua_client_type_mode = ('ua_client_type', lambda x: statistics.mode(list(x))),
    ua_client_type_len = ('ua_client_type', lambda x: len(set(x))),
    ua_os_mode = ('ua_os', lambda x: statistics.mode(list(x))),
    ua_os_len = ('ua_os', lambda x: len(set(x))),
    ua_client_name_mode = ('ua_client_name', lambda x: statistics.mode(list(x))),
    ua_client_name_len = ('ua_client_name', lambda x: len(set(x))),
    rutube_video_id_mode = ('rutube_video_id', lambda x: statistics.mode(list(x))),
    rutube_video_id_len = ('rutube_video_id', lambda x: len(set(x))),
    category_mode = ('category', lambda x: statistics.mode(list(x))),
    category_len = ('category', lambda x: len(set(x))),
    author_id_mode = ('author_id', lambda x: statistics.mode(list(x))),
    author_id_len = ('author_id', lambda x: len(set(x))),
    timezone_mode = ('timezone', lambda x: statistics.mode(list(x))),
    timezone_last = ('timezone', 'last'),
    lat = ('lat', 'last'),
    lon = ('lon', 'last'),
    total_watchtime_mean = ('total_watchtime', 'mean'),
    total_watchtime_sum = ('total_watchtime', 'sum'),
    cnt_views_sum = ('cnt_views', 'sum'),
    cnt_views_mean = ('cnt_views', 'mean'),
    duration_sum = ('duration', 'sum'),
    duration_mean = ('duration', 'mean'),
    share_watchtime_duration_sum = ('share_watchtime_duration', 'sum'),
    share_watchtime_duration_mean = ('share_watchtime_duration', 'mean'),
    category_text = ('category', lambda x: ', '.join(set(x))),
    text_male_count = ('text_male_count', 'mean'),
    text_female_count = ('text_female_count', 'mean'),
    text_age_0_class_count = ('text_age_0_class_count', 'mean'),
    text_age_1_class_count = ('text_age_1_class_count', 'mean'),
    text_age_2_class_count = ('text_age_2_class_count', 'mean'),
    text_age_3_class_count = ('text_age_3_class_count', 'mean'),
)
data_group


CPU times: user 1min 7s, sys: 2.65 ms, total: 1min 7s
Wall time: 1min 7s


,region_mode,region_len,ua_device_type_mode,ua_device_type_len,ua_client_type_mode,ua_client_type_len,ua_os_mode,ua_os_len,ua_client_name_mode,ua_client_name_len,...,duration_mean,share_watchtime_duration_sum,share_watchtime_duration_mean,category_text,text_male_count,text_female_count,text_age_0_class_count,text_age_1_class_count,text_age_2_class_count,text_age_3_class_count
viewer_uid,,,,,,,,,,,,,,,,,,,,,
10000001,Moscow,1,desktop,1,browser,1,Windows,1,Chrome,1,...,80.955406,0.322252,0.016961,Television Shows,0.0,0.0,0.0,0.0,0.0,0.0
10000002,Moscow,1,tablet,1,mobile app,1,Android,1,Rutube,1,...,45.065353,0.350058,0.038895,"Movies, Serials, Various",0.0,0.0,0.0,0.0,0.0,0.0
10000004,Moscow,3,smartphone,2,mobile app,1,Android,1,Rutube,1,...,37.836254,0.605133,0.050428,"Serials, Education, Science, Technology and In...",0.0,0.0,0.0,0.0,0.0,0.0
10000005,Tatarstan Republic,1,desktop,1,browser,1,Windows,1,Yandex Browser,1,...,8.751534,0.114266,0.114266,Technology and Internet,0.0,0.0,0.0,0.0,0.0,0.0
10000006,Sverdlovsk Oblast,1,smartphone,1,mobile app,1,Android,1,Rutube,1,...,19.697083,0.050769,0.050769,Anime,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11140828,St.-Petersburg,1,tablet,1,browser,1,Android,1,Firefox Mobile,1,...,6.347000,0.157555,0.157555,Education,0.0,0.0,0.0,0.0,0.0,0.0
11140869,St.-Petersburg,1,tablet,1,browser,1,Android,1,Firefox Mobile,1,...,6.347000,0.157555,0.157555,Education,0.0,0.0,0.0,0.0,0.0,0.0
11140872,Kaliningrad Oblast,1,desktop,1,browser,1,Windows,1,Microsoft Edge,1,...,6.270721,0.638663,0.159666,"Entertainment, Comedy",0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
data_group.select_dtypes(include=['object']).columns


Index(['region_mode', 'ua_device_type_mode', 'ua_client_type_mode',
       'ua_os_mode', 'ua_client_name_mode', 'rutube_video_id_mode',
       'category_mode', 'category_text'],
      dtype='object')

In [32]:
# Кодируем катеориальные признаки кроме
encoder = OrdinalEncoder()
columns_df = data_group.select_dtypes(include=['object']).columns[:-1]
df_encoded = encoder.fit_transform(data_group[columns_df])
df_encoded = pd.DataFrame(df_encoded, columns=columns_df)
data_group = data_group.reset_index()
df_encoded['viewer_uid'] = data_group['viewer_uid']

data_group = data_group.drop(columns_df, axis=1).merge(df_encoded, on=['viewer_uid'], how='inner')
data_group


,viewer_uid,region_len,ua_device_type_len,ua_client_type_len,ua_os_len,ua_client_name_len,rutube_video_id_len,category_len,author_id_mode,author_id_len,...,text_age_1_class_count,text_age_2_class_count,text_age_3_class_count,region_mode,ua_device_type_mode,ua_client_type_mode,ua_os_mode,ua_client_name_mode,rutube_video_id_mode,category_mode
0,10000001,1,1,1,1,1,16,1,1009257,1,...,0.0,0.0,0.0,54.0,0.0,1.0,16.0,8.0,15826.0,36.0
1,10000002,1,1,1,1,1,9,3,1065557,2,...,0.0,0.0,0.0,54.0,2.0,2.0,0.0,40.0,13613.0,33.0
2,10000004,3,2,1,1,1,11,4,1010000,1,...,0.0,0.0,0.0,54.0,1.0,2.0,0.0,40.0,2690.0,32.0
3,10000005,1,1,1,1,1,1,1,1025447,1,...,0.0,0.0,0.0,94.0,0.0,1.0,16.0,48.0,961.0,35.0
4,10000006,1,1,1,1,1,1,1,1028890,1,...,0.0,0.0,0.0,91.0,1.0,2.0,0.0,40.0,33441.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180007,11140828,1,1,1,1,1,1,1,1011978,1,...,0.0,0.0,0.0,86.0,2.0,1.0,0.0,20.0,32107.0,12.0
180008,11140869,1,1,1,1,1,1,1,1011978,1,...,0.0,0.0,0.0,86.0,2.0,1.0,0.0,20.0,32107.0,12.0
180009,11140872,1,1,1,1,1,4,2,1007052,2,...,0.0,0.0,0.0,26.0,0.0,1.0,16.0,30.0,32798.0,13.0
180010,11140875,1,1,1,1,1,1,1,1011978,1,...,0.0,0.0,0.0,54.0,2.0,1.0,0.0,20.0,32107.0,12.0


In [33]:
# Содиденям закодированные признаки
df_final = df_final.merge(data_group, on='viewer_uid', how='inner')


In [34]:
del data_group
del df_final_2


In [35]:
data.isna().sum().sum()


0

In [36]:
df_lemm_text = data.groupby(['viewer_uid'])['lemm_text'].apply(lambda x:  ', '.join(x)).reset_index()
df_lemm_text['lemm_text'] = df_lemm_text['lemm_text'].apply(lambda x: ' '.join(list(set(x.replace(',', '').split()))))
df_lemm_text['lemm_text']


0          выпуск битва сезон экстрасенс финал новый реванш
1         серия теория сериал сезон возмездие взрыв путь...
2         расчет на вечерний кенгуру гдр сергей гоминид ...
3                                     включать россия в как
4                            к бессмертие серия путешествие
                                ...                        
180007               или на снимать что камера айфон блогер
180008               или на снимать что камера айфон блогер
180009    на замораживать поиск для пранк нарываться с л...
180010               или на снимать что камера айфон блогер
180011               или на снимать что камера айфон блогер
Name: lemm_text, Length: 180012, dtype: object

In [37]:
df_final = df_final.merge(df_lemm_text, on='viewer_uid', how='inner')


In [38]:
df_target['sex'] = (df_target['sex'] == 'male') * 1
df_target = df_target.drop('age', axis=1)


In [39]:
df_final = df_final.merge(df_target, on='viewer_uid', how='inner')


In [40]:
# Сохраняем финальный датасет
df_final.to_parquet('data/for_train.pqt', engine='pyarrow')


/home/ruslan/rutube/venv/lib/python3.10/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
